Four different VGG Comvolutional Network are used below to test the effect of dropout on the train set accuracy as well a validation set accuracy. The data set used is cifar-10

Models:
1. not dropout
2. increasing dropout after convolutional layers (0.2 to 0.5) and no dropout in fully connected layers
3. increasing dropout after convolutional layers (0.1 to 0.4) and no dropout in fully connected layers
4. dropout applied only on fully connected layers 
5. increasing dropout after convolutional layers and fully connected layers

In [5]:
# not dropout used
import keras
from keras.models import Sequential
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.datasets import cifar10
from keras import regularizers
from keras.callbacks import LearningRateScheduler
import tensorflow as tf
import numpy as np
print("........")
print(tf.test.gpu_device_name())
print("........")
def lr_schedule(epoch):
    lrate = 0.001
    if epoch > 80:
        lrate = 0.00005
    elif epoch > 60:
        lrate = 0.0001
    elif epoch > 40:
        lrate = 0.0002
    elif epoch > 20:
        lrate = 0.0005        
    return lrate

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_test = x_train / 255, x_test / 255

#z-score


num_classes = 10
y_train = np_utils.to_categorical(y_train,num_classes)
y_test = np_utils.to_categorical(y_test,num_classes)

weight_decay = 1e-4
model = Sequential()
model.add(Conv2D(32, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l2(weight_decay), input_shape=x_train.shape[1:]))
model.add(BatchNormalization())
model.add(Conv2D(32, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(BatchNormalization())
model.add(Conv2D(64, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))


model.add(Conv2D(128, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(BatchNormalization())
model.add(Conv2D(128, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(BatchNormalization())
model.add(Conv2D(128, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(layers.Conv2D(256, (3, 3), padding='same', activation='relu', kernel_initializer='he_uniform'))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(256, (3, 3), padding='same', activation='relu', kernel_initializer='he_uniform'))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(256, (3, 3), padding='same', activation='relu', kernel_initializer='he_uniform'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))

model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(layers.BatchNormalization())
model.add(Dense(1024, activation='relu'))
model.add(layers.BatchNormalization())
model.add(Dense(num_classes, activation='softmax'))

model.summary()


#data augmentation
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    )
datagen.fit(x_train)

#training
batch_size = 64

opt_rms = tf.keras.optimizers.RMSprop(lr=0.001,decay=1e-6)
model.compile(loss='categorical_crossentropy', optimizer=opt_rms, metrics=['accuracy'])
model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),\
                    steps_per_epoch=x_train.shape[0] // batch_size,epochs=100,\
                    verbose=1,validation_data=(x_test,y_test),callbacks=[LearningRateScheduler(lr_schedule)])

........
/device:GPU:0
........


2021-12-07 19:23:02.941877: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2021-12-07 19:23:02.941902: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_20 (Conv2D)          (None, 32, 32, 32)        896       
                                                                 
 batch_normalization_20 (Bat  (None, 32, 32, 32)       128       
 chNormalization)                                                
                                                                 
 conv2d_21 (Conv2D)          (None, 32, 32, 32)        9248      
                                                                 
 batch_normalization_21 (Bat  (None, 32, 32, 32)       128       
 chNormalization)                                                
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 16, 16, 32)       0         
 2D)                                                             
                                                      

/var/folders/_3/stf971r13njfhfsz80m7bqwc0000gn/T/ipykernel_1438/218476954.py:130: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),\


Epoch 1/100


2021-12-07 19:23:04.245499: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


781/781 [==============================] - ETA: 0s - loss: 1.7288 - accuracy: 0.4325

2021-12-07 19:23:32.431943: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


781/781 [==============================] - 32s 38ms/step - loss: 1.7288 - accuracy: 0.4325 - val_loss: 1.5780 - val_accuracy: 0.5388 - lr: 0.0010
Epoch 2/100
781/781 [==============================] - 29s 37ms/step - loss: 1.1954 - accuracy: 0.6146 - val_loss: 1.5732 - val_accuracy: 0.5647 - lr: 0.0010
Epoch 3/100
781/781 [==============================] - 29s 37ms/step - loss: 1.0096 - accuracy: 0.6864 - val_loss: 1.1219 - val_accuracy: 0.6645 - lr: 0.0010
Epoch 4/100
781/781 [==============================] - 29s 37ms/step - loss: 0.9081 - accuracy: 0.7287 - val_loss: 0.9688 - val_accuracy: 0.7175 - lr: 0.0010
Epoch 5/100
781/781 [==============================] - 29s 37ms/step - loss: 0.8433 - accuracy: 0.7509 - val_loss: 0.7871 - val_accuracy: 0.7650 - lr: 0.0010
Epoch 6/100
781/781 [==============================] - 30s 38ms/step - loss: 0.7870 - accuracy: 0.7693 - val_loss: 0.9512 - val_accuracy: 0.7281 - lr: 0.0010
Epoch 7/100
781/781 [==============================] - 29s 37ms/

Epoch 52/100
781/781 [==============================] - 29s 37ms/step - loss: 0.2217 - accuracy: 0.9626 - val_loss: 0.4858 - val_accuracy: 0.8975 - lr: 2.0000e-04
Epoch 53/100
781/781 [==============================] - 29s 37ms/step - loss: 0.2175 - accuracy: 0.9648 - val_loss: 0.4796 - val_accuracy: 0.9003 - lr: 2.0000e-04
Epoch 54/100
781/781 [==============================] - 29s 38ms/step - loss: 0.2149 - accuracy: 0.9648 - val_loss: 0.4964 - val_accuracy: 0.8949 - lr: 2.0000e-04
Epoch 55/100
781/781 [==============================] - 29s 37ms/step - loss: 0.2135 - accuracy: 0.9640 - val_loss: 0.5234 - val_accuracy: 0.8926 - lr: 2.0000e-04
Epoch 56/100
781/781 [==============================] - 30s 38ms/step - loss: 0.2086 - accuracy: 0.9655 - val_loss: 0.5089 - val_accuracy: 0.8944 - lr: 2.0000e-04
Epoch 57/100
781/781 [==============================] - 29s 37ms/step - loss: 0.2097 - accuracy: 0.9649 - val_loss: 0.4983 - val_accuracy: 0.8956 - lr: 2.0000e-04
Epoch 58/100
781/781 [

In [7]:
# increasing dropout after convolutional layers (0.2 to 0.5) and no dropout in fully connected layers
import keras
from keras.models import Sequential
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.datasets import cifar10
from keras import regularizers
from keras.callbacks import LearningRateScheduler
import tensorflow as tf
import numpy as np
print("........")
print(tf.test.gpu_device_name())
print("........")
def lr_schedule(epoch):
    lrate = 0.001
    if epoch > 80:
        lrate = 0.00005
    elif epoch > 60:
        lrate = 0.0001
    elif epoch > 40:
        lrate = 0.0002
    elif epoch > 20:
        lrate = 0.0005        
    return lrate

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_test = x_train / 255, x_test / 255

#z-score


num_classes = 10
y_train = np_utils.to_categorical(y_train,num_classes)
y_test = np_utils.to_categorical(y_test,num_classes)

weight_decay = 1e-4
model = Sequential()
model.add(Conv2D(32, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l2(weight_decay), input_shape=x_train.shape[1:]))
model.add(BatchNormalization())
model.add(Conv2D(32, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(layers.Dropout(0.2))

model.add(Conv2D(64, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(BatchNormalization())
model.add(Conv2D(64, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(layers.Dropout(0.3))


model.add(Conv2D(128, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(BatchNormalization())
model.add(Conv2D(128, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(BatchNormalization())
model.add(Conv2D(128, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(layers.Dropout(0.4))

model.add(layers.Conv2D(256, (3, 3), padding='same', activation='relu', kernel_initializer='he_uniform'))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(256, (3, 3), padding='same', activation='relu', kernel_initializer='he_uniform'))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(256, (3, 3), padding='same', activation='relu', kernel_initializer='he_uniform'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.5))

model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(layers.BatchNormalization())
model.add(Dense(1024, activation='relu'))
model.add(layers.BatchNormalization())
model.add(Dense(num_classes, activation='softmax'))

model.summary()


#data augmentation
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    )
datagen.fit(x_train)

#training
batch_size = 64

opt_rms = tf.keras.optimizers.RMSprop(lr=0.001,decay=1e-6)
model.compile(loss='categorical_crossentropy', optimizer=opt_rms, metrics=['accuracy'])
model.fit(datagen.flow(x_train, y_train, batch_size=batch_size),\
                    steps_per_epoch=x_train.shape[0] // batch_size,epochs=100,\
                    verbose=1,validation_data=(x_test,y_test),callbacks=[LearningRateScheduler(lr_schedule)])

........
/device:GPU:0
........


2021-12-07 20:12:31.314718: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2021-12-07 20:12:31.314744: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_40 (Conv2D)          (None, 32, 32, 32)        896       
                                                                 
 batch_normalization_44 (Bat  (None, 32, 32, 32)       128       
 chNormalization)                                                
                                                                 
 conv2d_41 (Conv2D)          (None, 32, 32, 32)        9248      
                                                                 
 batch_normalization_45 (Bat  (None, 32, 32, 32)       128       
 chNormalization)                                                
                                                                 
 max_pooling2d_16 (MaxPoolin  (None, 16, 16, 32)       0         
 g2D)                                                            
                                                      

2021-12-07 20:12:32.605596: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


780/781 [============================>.] - ETA: 0s - loss: 1.8665 - accuracy: 0.3876

2021-12-07 20:13:02.653952: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


781/781 [==============================] - 34s 39ms/step - loss: 1.8664 - accuracy: 0.3876 - val_loss: 2.3577 - val_accuracy: 0.3822 - lr: 0.0010
Epoch 2/100
781/781 [==============================] - 31s 39ms/step - loss: 1.3507 - accuracy: 0.5582 - val_loss: 1.4362 - val_accuracy: 0.5699 - lr: 0.0010
Epoch 3/100
781/781 [==============================] - 30s 39ms/step - loss: 1.1528 - accuracy: 0.6306 - val_loss: 1.2486 - val_accuracy: 0.6342 - lr: 0.0010
Epoch 4/100
781/781 [==============================] - 30s 38ms/step - loss: 1.0480 - accuracy: 0.6730 - val_loss: 1.0211 - val_accuracy: 0.6867 - lr: 0.0010
Epoch 5/100
781/781 [==============================] - 30s 38ms/step - loss: 0.9801 - accuracy: 0.7016 - val_loss: 1.1208 - val_accuracy: 0.6784 - lr: 0.0010
Epoch 6/100
781/781 [==============================] - 30s 38ms/step - loss: 0.9253 - accuracy: 0.7258 - val_loss: 0.9619 - val_accuracy: 0.7060 - lr: 0.0010
Epoch 7/100
781/781 [==============================] - 30s 39ms/

Epoch 52/100
781/781 [==============================] - 31s 39ms/step - loss: 0.3821 - accuracy: 0.9076 - val_loss: 0.4458 - val_accuracy: 0.8939 - lr: 2.0000e-04
Epoch 53/100
781/781 [==============================] - 31s 40ms/step - loss: 0.3783 - accuracy: 0.9093 - val_loss: 0.4577 - val_accuracy: 0.8886 - lr: 2.0000e-04
Epoch 54/100
781/781 [==============================] - 31s 40ms/step - loss: 0.3771 - accuracy: 0.9090 - val_loss: 0.4502 - val_accuracy: 0.8924 - lr: 2.0000e-04
Epoch 55/100
781/781 [==============================] - 31s 39ms/step - loss: 0.3771 - accuracy: 0.9094 - val_loss: 0.4491 - val_accuracy: 0.8925 - lr: 2.0000e-04
Epoch 56/100
781/781 [==============================] - 31s 39ms/step - loss: 0.3674 - accuracy: 0.9122 - val_loss: 0.4587 - val_accuracy: 0.8913 - lr: 2.0000e-04
Epoch 57/100
781/781 [==============================] - 31s 40ms/step - loss: 0.3740 - accuracy: 0.9097 - val_loss: 0.4547 - val_accuracy: 0.8873 - lr: 2.0000e-04
Epoch 58/100
781/781 [

In [8]:
# increasing dropout after convolutional layers (0.1 to 0.4) and no dropout in fully connected layers
import keras
from keras.models import Sequential
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.datasets import cifar10
from keras import regularizers
from keras.callbacks import LearningRateScheduler
import tensorflow as tf
import numpy as np
print("........")
print(tf.test.gpu_device_name())
print("........")
def lr_schedule(epoch):
    lrate = 0.001
    if epoch > 80:
        lrate = 0.00005
    elif epoch > 60:
        lrate = 0.0001
    elif epoch > 40:
        lrate = 0.0002
    elif epoch > 20:
        lrate = 0.0005        
    return lrate

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_test = x_train / 255, x_test / 255

num_classes = 10
y_train = np_utils.to_categorical(y_train,num_classes)
y_test = np_utils.to_categorical(y_test,num_classes)


weight_decay = 1e-4
model = Sequential()
model.add(Conv2D(32, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l2(weight_decay), input_shape=x_train.shape[1:]))
model.add(BatchNormalization())
model.add(Conv2D(32, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(layers.Dropout(0.1))

model.add(Conv2D(64, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(BatchNormalization())
model.add(Conv2D(64, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(layers.Dropout(0.2))


model.add(Conv2D(128, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(BatchNormalization())
model.add(Conv2D(128, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(BatchNormalization())
model.add(Conv2D(128, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(layers.Dropout(0.3))

model.add(layers.Conv2D(256, (3, 3), padding='same', activation='relu', kernel_initializer='he_uniform'))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(256, (3, 3), padding='same', activation='relu', kernel_initializer='he_uniform'))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(256, (3, 3), padding='same', activation='relu', kernel_initializer='he_uniform'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.4))

model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(layers.BatchNormalization())
model.add(Dense(1024, activation='relu'))
model.add(layers.BatchNormalization())
model.add(Dense(num_classes, activation='softmax'))

model.summary()


#data augmentation
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    )
datagen.fit(x_train)

#training
batch_size = 64

opt_rms = tf.keras.optimizers.RMSprop(lr=0.001,decay=1e-6)
model.compile(loss='categorical_crossentropy', optimizer=opt_rms, metrics=['accuracy'])
model.fit(datagen.flow(x_train, y_train, batch_size=batch_size),\
                    steps_per_epoch=x_train.shape[0] // batch_size,epochs=100,\
                    verbose=1,validation_data=(x_test,y_test),callbacks=[LearningRateScheduler(lr_schedule)])

........
/device:GPU:0
........


2021-12-07 21:08:31.727725: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2021-12-07 21:08:31.727753: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_50 (Conv2D)          (None, 32, 32, 32)        896       
                                                                 
 batch_normalization_56 (Bat  (None, 32, 32, 32)       128       
 chNormalization)                                                
                                                                 
 conv2d_51 (Conv2D)          (None, 32, 32, 32)        9248      
                                                                 
 batch_normalization_57 (Bat  (None, 32, 32, 32)       128       
 chNormalization)                                                
                                                                 
 max_pooling2d_20 (MaxPoolin  (None, 16, 16, 32)       0         
 g2D)                                                            
                                                      

2021-12-07 21:08:33.225656: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


781/781 [==============================] - ETA: 0s - loss: 1.8052 - accuracy: 0.4053

2021-12-07 21:09:04.638483: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


781/781 [==============================] - 36s 41ms/step - loss: 1.8052 - accuracy: 0.4053 - val_loss: 1.5611 - val_accuracy: 0.5443 - lr: 0.0010
Epoch 2/100
781/781 [==============================] - 33s 42ms/step - loss: 1.2842 - accuracy: 0.5793 - val_loss: 1.4839 - val_accuracy: 0.5363 - lr: 0.0010
Epoch 3/100
781/781 [==============================] - 31s 40ms/step - loss: 1.0890 - accuracy: 0.6542 - val_loss: 1.0696 - val_accuracy: 0.6542 - lr: 0.0010
Epoch 4/100
781/781 [==============================] - 33s 42ms/step - loss: 0.9833 - accuracy: 0.6981 - val_loss: 1.0072 - val_accuracy: 0.6951 - lr: 0.0010
Epoch 5/100
781/781 [==============================] - 32s 41ms/step - loss: 0.9170 - accuracy: 0.7225 - val_loss: 0.9989 - val_accuracy: 0.7066 - lr: 0.0010
Epoch 6/100
781/781 [==============================] - 31s 39ms/step - loss: 0.8574 - accuracy: 0.7456 - val_loss: 0.9872 - val_accuracy: 0.7319 - lr: 0.0010
Epoch 7/100
781/781 [==============================] - 31s 39ms/

Epoch 52/100
781/781 [==============================] - 31s 40ms/step - loss: 0.3283 - accuracy: 0.9274 - val_loss: 0.4485 - val_accuracy: 0.8977 - lr: 2.0000e-04
Epoch 53/100
781/781 [==============================] - 32s 40ms/step - loss: 0.3271 - accuracy: 0.9265 - val_loss: 0.4473 - val_accuracy: 0.8996 - lr: 2.0000e-04
Epoch 54/100
781/781 [==============================] - 31s 39ms/step - loss: 0.3225 - accuracy: 0.9283 - val_loss: 0.4558 - val_accuracy: 0.8960 - lr: 2.0000e-04
Epoch 55/100
781/781 [==============================] - 31s 40ms/step - loss: 0.3143 - accuracy: 0.9329 - val_loss: 0.4749 - val_accuracy: 0.8878 - lr: 2.0000e-04
Epoch 56/100
781/781 [==============================] - 31s 40ms/step - loss: 0.3138 - accuracy: 0.9311 - val_loss: 0.4755 - val_accuracy: 0.8901 - lr: 2.0000e-04
Epoch 57/100
781/781 [==============================] - 31s 40ms/step - loss: 0.3132 - accuracy: 0.9313 - val_loss: 0.4393 - val_accuracy: 0.8988 - lr: 2.0000e-04
Epoch 58/100
781/781 [

In [9]:
# dropout applied only on fully connected layers 
# this is what the original paper on dropout proposed:
#      https://arxiv.org/pdf/1207.0580.pdf
import keras
from keras.models import Sequential
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.datasets import cifar10
from keras import regularizers
from keras.callbacks import LearningRateScheduler
import tensorflow as tf
import numpy as np
print("........")
print(tf.test.gpu_device_name())
print("........")
def lr_schedule(epoch):
    lrate = 0.001
    if epoch > 80:
        lrate = 0.00005
    elif epoch > 60:
        lrate = 0.0001
    elif epoch > 40:
        lrate = 0.0002
    elif epoch > 20:
        lrate = 0.0005        
    return lrate

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_test = x_train / 255, x_test / 255

#z-score


num_classes = 10
y_train = np_utils.to_categorical(y_train,num_classes)
y_test = np_utils.to_categorical(y_test,num_classes)

weight_decay = 1e-4
model = Sequential()
model.add(Conv2D(32, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l2(weight_decay), input_shape=x_train.shape[1:]))
model.add(BatchNormalization())
model.add(Conv2D(32, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(BatchNormalization())
model.add(Conv2D(64, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(128, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(BatchNormalization())
model.add(Conv2D(128, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(BatchNormalization())
model.add(Conv2D(128, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(layers.Conv2D(256, (3, 3), padding='same', activation='relu', kernel_initializer='he_uniform'))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(256, (3, 3), padding='same', activation='relu', kernel_initializer='he_uniform'))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(256, (3, 3), padding='same', activation='relu', kernel_initializer='he_uniform'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))

model.add(Flatten())
model.add(layers.Dropout(0.5))
model.add(Dense(1024, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.5))
model.add(Dense(1024, activation='relu'))
model.add(layers.BatchNormalization())
model.add(Dense(num_classes, activation='softmax'))

model.summary()


#data augmentation
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    )
datagen.fit(x_train)

#training
batch_size = 64

opt_rms = tf.keras.optimizers.RMSprop(lr=0.001,decay=1e-6)
model.compile(loss='categorical_crossentropy', optimizer=opt_rms, metrics=['accuracy'])
model.fit(datagen.flow(x_train, y_train, batch_size=batch_size),\
                    steps_per_epoch=x_train.shape[0] // batch_size,epochs=100,\
                    verbose=1,validation_data=(x_test,y_test),callbacks=[LearningRateScheduler(lr_schedule)])

........
/device:GPU:0
........


2021-12-07 22:01:50.888158: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2021-12-07 22:01:50.888184: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_60 (Conv2D)          (None, 32, 32, 32)        896       
                                                                 
 batch_normalization_68 (Bat  (None, 32, 32, 32)       128       
 chNormalization)                                                
                                                                 
 conv2d_61 (Conv2D)          (None, 32, 32, 32)        9248      
                                                                 
 batch_normalization_69 (Bat  (None, 32, 32, 32)       128       
 chNormalization)                                                
                                                                 
 max_pooling2d_24 (MaxPoolin  (None, 16, 16, 32)       0         
 g2D)                                                            
                                                      

2021-12-07 22:01:52.154336: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


780/781 [============================>.] - ETA: 0s - loss: 1.9336 - accuracy: 0.3796

2021-12-07 22:02:23.566280: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


781/781 [==============================] - 36s 42ms/step - loss: 1.9336 - accuracy: 0.3796 - val_loss: 1.9202 - val_accuracy: 0.4632 - lr: 0.0010
Epoch 2/100
781/781 [==============================] - 31s 40ms/step - loss: 1.3423 - accuracy: 0.5631 - val_loss: 1.1862 - val_accuracy: 0.6191 - lr: 0.0010
Epoch 3/100
781/781 [==============================] - 31s 39ms/step - loss: 1.0903 - accuracy: 0.6557 - val_loss: 1.4038 - val_accuracy: 0.5671 - lr: 0.0010
Epoch 4/100
781/781 [==============================] - 31s 39ms/step - loss: 0.9588 - accuracy: 0.7013 - val_loss: 1.8892 - val_accuracy: 0.5499 - lr: 0.0010
Epoch 5/100
781/781 [==============================] - 31s 39ms/step - loss: 0.8740 - accuracy: 0.7370 - val_loss: 0.8670 - val_accuracy: 0.7429 - lr: 0.0010
Epoch 6/100
781/781 [==============================] - 30s 39ms/step - loss: 0.8158 - accuracy: 0.7615 - val_loss: 0.9448 - val_accuracy: 0.7334 - lr: 0.0010
Epoch 7/100
781/781 [==============================] - 31s 40ms/

Epoch 52/100
781/781 [==============================] - 30s 39ms/step - loss: 0.2489 - accuracy: 0.9570 - val_loss: 0.4769 - val_accuracy: 0.8948 - lr: 2.0000e-04
Epoch 53/100
781/781 [==============================] - 30s 39ms/step - loss: 0.2467 - accuracy: 0.9568 - val_loss: 0.4546 - val_accuracy: 0.9021 - lr: 2.0000e-04
Epoch 54/100
781/781 [==============================] - 30s 39ms/step - loss: 0.2424 - accuracy: 0.9574 - val_loss: 0.4658 - val_accuracy: 0.9021 - lr: 2.0000e-04
Epoch 55/100
781/781 [==============================] - 30s 39ms/step - loss: 0.2420 - accuracy: 0.9587 - val_loss: 0.4732 - val_accuracy: 0.8973 - lr: 2.0000e-04
Epoch 56/100
781/781 [==============================] - 30s 39ms/step - loss: 0.2378 - accuracy: 0.9585 - val_loss: 0.4858 - val_accuracy: 0.8949 - lr: 2.0000e-04
Epoch 57/100
781/781 [==============================] - 30s 38ms/step - loss: 0.2308 - accuracy: 0.9610 - val_loss: 0.4786 - val_accuracy: 0.8989 - lr: 2.0000e-04
Epoch 58/100
781/781 [

In [11]:
# additional 100 epochs of the previous model
def lr(epoch):
    lrate = 0.00002
    if epoch >= 25:
        lrate = 0.00001     
    return lrate

model.compile(loss='categorical_crossentropy', optimizer=opt_rms, metrics=['accuracy'])
model.fit(datagen.flow(x_train, y_train, batch_size=batch_size),\
                    steps_per_epoch=x_train.shape[0] // batch_size,epochs=100,\
                    verbose=1,validation_data=(x_test,y_test),callbacks=[LearningRateScheduler(lr)])

Epoch 1/100


2021-12-07 22:54:12.652394: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


780/781 [============================>.] - ETA: 0s - loss: 0.1457 - accuracy: 0.9829

2021-12-07 22:54:44.796663: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


781/781 [==============================] - 36s 41ms/step - loss: 0.1457 - accuracy: 0.9830 - val_loss: 0.4826 - val_accuracy: 0.9075 - lr: 2.0000e-05
Epoch 2/100
781/781 [==============================] - 31s 39ms/step - loss: 0.1449 - accuracy: 0.9834 - val_loss: 0.4825 - val_accuracy: 0.9089 - lr: 2.0000e-05
Epoch 3/100
781/781 [==============================] - 30s 39ms/step - loss: 0.1432 - accuracy: 0.9839 - val_loss: 0.4880 - val_accuracy: 0.9089 - lr: 2.0000e-05
Epoch 4/100
781/781 [==============================] - 31s 39ms/step - loss: 0.1466 - accuracy: 0.9829 - val_loss: 0.4916 - val_accuracy: 0.9074 - lr: 2.0000e-05
Epoch 5/100
781/781 [==============================] - 31s 40ms/step - loss: 0.1423 - accuracy: 0.9839 - val_loss: 0.4821 - val_accuracy: 0.9093 - lr: 2.0000e-05
Epoch 6/100
781/781 [==============================] - 31s 40ms/step - loss: 0.1450 - accuracy: 0.9828 - val_loss: 0.4798 - val_accuracy: 0.9109 - lr: 2.0000e-05
Epoch 7/100
781/781 [===================

In [3]:
# increasing dropout after convolutional layers and fully connected layers
import keras
from keras.models import Sequential
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.datasets import cifar10
from keras import regularizers
from keras.callbacks import LearningRateScheduler
import tensorflow as tf
import numpy as np
print("........")
print(tf.test.gpu_device_name())
print("........")
def lr_schedule(epoch):
    lrate = 0.001
    if epoch > 80:
        lrate = 0.00005
    elif epoch > 60:
        lrate = 0.0001
    elif epoch > 40:
        lrate = 0.0002
    elif epoch > 20:
        lrate = 0.0005        
    return lrate

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_test = x_train / 255, x_test / 255

#z-score


num_classes = 10
y_train = np_utils.to_categorical(y_train,num_classes)
y_test = np_utils.to_categorical(y_test,num_classes)

weight_decay = 1e-4
model = Sequential()
model.add(Conv2D(32, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l2(weight_decay), input_shape=x_train.shape[1:]))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Conv2D(32, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Conv2D(64, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(128, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(BatchNormalization())
model.add(Dropout(0.4))
model.add(Conv2D(128, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(BatchNormalization())
model.add(Dropout(0.4))
model.add(Conv2D(128, (3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(BatchNormalization())
model.add(Dropout(0.4))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(256, (3, 3), padding='same', activation='relu', kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Conv2D(256, (3, 3), padding='same', activation='relu', kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Conv2D(256, (3, 3), padding='same', activation='relu', kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(1024, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(1024, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(num_classes, activation='softmax'))

model.summary()


#data augmentation
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    )
datagen.fit(x_train)

#training
batch_size = 64

opt_rms = tf.keras.optimizers.RMSprop(lr=0.001,decay=1e-6)
model.compile(loss='categorical_crossentropy', optimizer=opt_rms, metrics=['accuracy'])
model.fit(datagen.flow(x_train, y_train, batch_size=batch_size),\
                    steps_per_epoch=x_train.shape[0] // batch_size,epochs=100,\
                    verbose=1,validation_data=(x_test,y_test),callbacks=[LearningRateScheduler(lr_schedule)])

........
/device:GPU:0
........


2021-12-08 13:50:24.919739: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2021-12-08 13:50:24.919761: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 32, 32, 32)        896       
                                                                 
 batch_normalization_2 (Batc  (None, 32, 32, 32)       128       
 hNormalization)                                                 
                                                                 
 dropout (Dropout)           (None, 32, 32, 32)        0         
                                                                 
 conv2d_3 (Conv2D)           (None, 32, 32, 32)        9248      
                                                                 
 batch_normalization_3 (Batc  (None, 32, 32, 32)       128       
 hNormalization)                                                 
                                                                 
 dropout_1 (Dropout)         (None, 32, 32, 32)       

/Users/fanrongqi/miniforge3/envs/tf/lib/python3.9/site-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)
2021-12-08 13:50:25.705341: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Epoch 1/100


2021-12-08 13:50:26.429749: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


781/781 [==============================] - ETA: 0s - loss: 2.1328 - accuracy: 0.2893

2021-12-08 13:50:56.299195: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


781/781 [==============================] - 34s 41ms/step - loss: 2.1328 - accuracy: 0.2893 - val_loss: 2.7007 - val_accuracy: 0.1977 - lr: 0.0010
Epoch 2/100
781/781 [==============================] - 31s 40ms/step - loss: 1.6159 - accuracy: 0.4616 - val_loss: 5.4362 - val_accuracy: 0.3508 - lr: 0.0010
Epoch 3/100
781/781 [==============================] - 31s 40ms/step - loss: 1.4254 - accuracy: 0.5423 - val_loss: 1.2727 - val_accuracy: 0.5913 - lr: 0.0010
Epoch 4/100
781/781 [==============================] - 31s 40ms/step - loss: 1.2601 - accuracy: 0.6002 - val_loss: 2.7781 - val_accuracy: 0.5448 - lr: 0.0010
Epoch 5/100
781/781 [==============================] - 30s 39ms/step - loss: 1.1586 - accuracy: 0.6384 - val_loss: 1.3218 - val_accuracy: 0.5755 - lr: 0.0010
Epoch 6/100
781/781 [==============================] - 31s 39ms/step - loss: 1.0852 - accuracy: 0.6668 - val_loss: 1.0710 - val_accuracy: 0.6793 - lr: 0.0010
Epoch 7/100
781/781 [==============================] - 30s 39ms/

Epoch 52/100
781/781 [==============================] - 31s 40ms/step - loss: 0.5375 - accuracy: 0.8570 - val_loss: 0.5664 - val_accuracy: 0.8467 - lr: 2.0000e-04
Epoch 53/100
781/781 [==============================] - 32s 40ms/step - loss: 0.5362 - accuracy: 0.8567 - val_loss: 0.5696 - val_accuracy: 0.8444 - lr: 2.0000e-04
Epoch 54/100
781/781 [==============================] - 31s 39ms/step - loss: 0.5341 - accuracy: 0.8589 - val_loss: 0.5669 - val_accuracy: 0.8428 - lr: 2.0000e-04
Epoch 55/100
781/781 [==============================] - 396s 508ms/step - loss: 0.5293 - accuracy: 0.8604 - val_loss: 0.5566 - val_accuracy: 0.8507 - lr: 2.0000e-04
Epoch 56/100
781/781 [==============================] - 32s 41ms/step - loss: 0.5276 - accuracy: 0.8599 - val_loss: 0.6220 - val_accuracy: 0.8263 - lr: 2.0000e-04
Epoch 57/100
781/781 [==============================] - 32s 41ms/step - loss: 0.5192 - accuracy: 0.8625 - val_loss: 0.5760 - val_accuracy: 0.8435 - lr: 2.0000e-04
Epoch 58/100
781/781